In [19]:
import saspy
sas = saspy.SASsession()

Using SAS Config named: oda
SAS Connection established. Subprocess id is 2760



In [2]:
#IMPORTING MODULES
import pandas as pd
import numpy as np

In [21]:
%%SAS sas
libname l '/home/u49467254/CRM';

filename refile '/home/u49467254/loan_data_2007_2014.csv';

data l.loan (drop=var1 url desc);
infile refile dsd missover firstobs=2;
length purpose $18 loan_status $70 verification_status $20 emp_title $70 title $72 ;
input var1 id member_id loan_amnt funded_amnt funded_amnt_inv term $ int_rate installment grade $ sub_grade $ emp_title $ emp_length $
 home_ownership $ annual_inc verification_status $ issue_d $ loan_status $ pymnt_plan $ url $
 desc $ purpose $ title $ zip_code $ addr_state $ dti $ delinq_2yrs earliest_cr_line $ inq_last_6mths mths_since_last_delinq
 mths_since_last_record open_acc pub_rec revol_bal revol_util total_acc initial_list_status	$ out_prncp out_prncp_inv total_pymnt
 total_pymnt_inv total_rec_prncp total_rec_int total_rec_late_fee recoveries collection_recovery_fee last_pymnt_d $
 last_pymnt_amnt next_pymnt_d $ last_credit_pull_d $ collections_12_mths_ex_med mths_since_last_major_derog policy_code	
 application_type $	annual_inc_joint $ dti_joint $ verification_status_joint $ acc_now_delinq $	tot_coll_amt $ tot_cur_bal $ open_acc_6m $ open_il_6m $
 open_il_12m $ open_il_24m $ mths_since_rcnt_il $ total_bal_il $ il_util $ open_rv_12m $ open_rv_24m $ max_bal_bc $	all_util $ total_rev_hi_lim $
 inq_fi $ total_cu_tl $ inq_last_12m $
 ;
run;


In [22]:
%%SAS sas

ods pdf file="/home/u49467254/CRM/loan_for_cleaning.pdf";
title "First 5 Observations of loan for viewing";
proc print data= l.loan (obs=5);
var _all_;
run;
ods pdf close;


Obs,purpose,loan_status,verification_status,emp_title,title,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,issue_d,pymnt_plan,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
1,credit_card,Fully Paid,Verified,,Computer,1077501,1296599,5000,5000,4975,36 month,10.65,162.87,B,B2,10+ year,RENT,24000,Dec-11,n,860xx,AZ,27.65,0,Jan-85,1,.,.,3,0,13648,83.7,9,f,0.0,0.0,5861.07,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,,Jan-16,0,.,1,INDIVIDU,,,,0.0,,,,,,,,,,,,,,,,,
2,car,Charged Off,Source Verified,Ryder,bike,1077430,1314167,2500,2500,2500,60 month,15.27,59.83,C,C4,< 1 year,RENT,30000,Dec-11,n,309xx,GA,1.0,0,Apr-99,5,.,.,3,0,1687,9.4,4,f,0.0,0.0,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,,Sep-13,0,.,1,INDIVIDU,,,,0.0,,,,,,,,,,,,,,,,,
3,small_business,Fully Paid,Not Verified,,real estate business,1077175,1313524,2400,2400,2400,36 month,15.96,84.33,C,C5,10+ year,RENT,12252,Dec-11,n,606xx,IL,8.72,0,Nov-01,2,.,.,2,0,2956,98.5,10,f,0.0,0.0,3003.65,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,,Jan-16,0,.,1,INDIVIDU,,,,0.0,,,,,,,,,,,,,,,,,
4,other,Fully Paid,Source Verified,AIR RESOURCES BOARD,personel,1076863,1277178,10000,10000,10000,36 month,13.49,339.31,C,C1,10+ year,RENT,49200,Dec-11,n,917xx,CA,20.0,0,Feb-96,1,35,.,10,0,5598,21.0,37,f,0.0,0.0,12226.30,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,,Jan-15,0,.,1,INDIVIDU,,,,0.0,,,,,,,,,,,,,,,,,
5,other,Current,Source Verified,University Medical Group,Personal,1075358,1311748,3000,3000,3000,60 month,12.69,67.79,B,B5,1 year,RENT,80000,Dec-11,n,972xx,OR,17.94,0,Jan-96,0,38,.,15,0,27783,53.9,38,f,766.9,766.9,3242.17,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-16,67.79,Feb-16,Jan-16,0,.,1,INDIVIDU,,,,0.0,,,,,,,,,,,,,,,,,


date variables are issue_d, earliest_cr_line, last_pymnt_d, next_pymnt_d, and last_credit_pull_d ;

In [23]:
%%SAS sas

proc contents 
    data=l.loan
    out =data_structure(keep= name type); 
run;

In [24]:
%%SAS sas

%let varUvalue =  grade ;
proc sql;
select &varUvalue, total, total/sum(total) as proportion
from 
    (select distinct(&varUvalue) as &varUvalue, count(&varUvalue) as total
    from l.loan
    group by &varUvalue);
quit;
run;

grade,total,proportion
A,74867,0.160561
B,136929,0.293659
C,125293,0.268705
D,76888,0.164895
E,35757,0.076685
F,13229,0.028371
G,3322,0.007124


* var to be removed are verification_status_joint, total_cu_tl, inq_fi, all_util 
max_bal_bc,open_rv_12m,open_rv_24, open_acc_6m, open_il_6m, open_il_12m, open_il_24m,
mths_since_rcnt_il, total_bal_il,  il_util. 

* var that are actually integer and shuold be, are total_rev_hi_lim, emp_length, and term.


In [25]:
%%SAS sas

data loan_prep;
    set l.loan;
    drop verification_status_joint total_cu_tl inq_fi all_util 
          max_bal_bc open_rv_12m open_rv_24m open_acc_6m open_il_6m open_il_12m open_il_24m
          mths_since_rcnt_il total_bal_il il_util;

    total_rev_hi_lim_int = input(total_rev_hi_lim, 8. );
    
    if emp_length = '10+ years' then emp_length_int = 10;
    if emp_length = '< 1 year' then emp_length_int = 0;
    if emp_length = '' then emp_length_int = 0;
    if emp_length = '1 year' then emp_length_int = 1;
    if emp_length = '3 years' then emp_length_int = 3;
    if emp_length = '4 years' then emp_length_int = 4;
    if emp_length = '5 years' then emp_length_int = 5;
    if emp_length = '6 years' then emp_length_int = 6;
    if emp_length = '7 years' then emp_length_int = 7;
    if emp_length = '8 years' then emp_length_int = 8;
    if emp_length = '9 years' then emp_length_int = 9;
    if emp_length = '2 years' then emp_length_int = 2;
    if term = '36 month' then term_int = 0;
    if term = '60 month' then term_int = 1;
run;

* fixing date variables

In [26]:
%%SAS sas

%macro date_fix(DSin,date,DSout); 
data &DSout. (drop =temp temp2 month year);
set &DSin.;
temp = index(&date.,'-')-1;
temp2 = index(&date.,'-')+1;
month = compbl('1'||(upcase(substr(&date., 1, temp))));
year = compbl('20'||(substr(&date., temp2)));
&date._fix= input(compbl(month||year), date9.);
format &date._fix date9.;
run;
%mend;

%date_fix(loan_prep, issue_d, loan_prep);
%date_fix(loan_prep, earliest_cr_line, loan_prep);
%date_fix(loan_prep, last_pymnt_d, loan_prep);
%date_fix(loan_prep, next_pymnt_d, loan_prep);
%date_fix(loan_prep, last_credit_pull_d, loan_prep);


* creating new variables

In [27]:
%%SAS sas


data loan_date;
set loan_prep;
*Assuming 2017-12-01 as today()/reference date because loan data is too old.;
    mths_since_earliest_cr_line = intck('month',earliest_cr_line_fix,'01DEC2017'd);
    mths_since_issue_date = intck('month',issue_d_fix,'01DEC2017'd);
run;
 

In [28]:
%%SAS sas

proc univariate data=loan_date;
    var mths_since_earliest_cr_line;
run;

In [29]:
%%SAS sas

%let negatives = mths_since_earliest_cr_line;
%let negveTable = loan_date;
title "proportion of negative values in the variable &negatives.";
proc sql;
select (count(&negatives.) /( select count(&negatives.) from loan_date) ) as prop_negatives
from &negveTable.
where &negatives. < 0;
quit;

prop_negatives
0.56039


* considering negative values as old data we replace it with its max vale

In [30]:
%%SAS sas

proc sql;
create table loan_miss1 as 
select *, (case 
                when mths_since_earliest_cr_line < 0 
                then max(mths_since_earliest_cr_line) 
                else mths_since_earliest_cr_line end
            ) as mths_since_earliest_cr_line_fix
from loan_date;
quit;

* replacing the following variable's missing value with 0 for they are of small proportion.

In [31]:
%%SAS sas

proc stdize data= loan_miss1
            reponly missing = 0
            out = loan_miss2;
    var total_acc pub_rec open_acc inq_last_6mths delinq_2yrs emp_length_int;
run;

proc stdize data= loan_miss2
            reponly method = mean
            out = loan_miss3;
    var annual_inc;
run;

proc sql;
create table loan_miss4 as 
select *, (case when total_rev_hi_lim_int is null then funded_amnt else total_rev_hi_lim_int end) as total_rev_hi_lim_fix
from loan_miss3;
quit;

proc means data=loan_miss4 nmiss;
    var annual_inc total_acc pub_rec open_acc inq_last_6mths delinq_2yrs
    emp_length_int total_rev_hi_lim_fix;
run;

Variable,N Miss
annual_inc total_acc pub_rec open_acc inq_last_6mths delinq_2yrs emp_length_int total_rev_hi_lim_fix,0 0 0 0 0 0 0 0


* working with target variable.

In [35]:
%%SAS sas

proc sql;
select uvalues, total, total/sum(total) as proportion
from 
    (select distinct(loan_status) as uvalues, count(loan_status) as total
    from loan_miss4
    group by loan_status);
quit;



uvalues,total,proportion
Charged Off,42475,0.091092
Current,224226,0.480878
Default,832,0.001784
Does not meet the credit policy. Status:Charged Off,761,0.001632
Does not meet the credit policy. Status:Fully Paid,1988,0.004263
Fully Paid,184739,0.396193
In Grace Period,3146,0.006747
Late (16-30 days),1218,0.002612
Late (31-120 days),6900,0.014798


In [34]:
%%SAS sas

data l.loan_clean;
set loan_miss4 (drop=total_rev_hi_lim total_rev_hi_lim_int mths_since_earliest_cr_line);
bad = 0;
if loan_status = 'Charged Off' then bad = 1;
if loan_status = 'Default' then bad = 1;
if loan_status = 'Does not meet the credit policy. Status:Charged Off' then bad = 1;
if loan_status = 'Late (31-120 days)' then bad = 1;
run;

*bad = 0 if defaulted and bad = 1 otherwise;
 also creating check point by saving data in memory;

In [36]:
%%SAS sas
proc export data=l.loan_clean 
    dbms=xlsx 
    outfile= '/home/u49467254/CRM/loan_clean.xlsx' 
    replace;
    sheet="First Data";
run;